## NovaSAR-1 CEOS-ARD RTC Gamma0 data <img align="right" src="../../resources/csiro_easi_logo.png">

#### Index
- [Overview](#Overview)
- [Setup (imports, defaults, dask, odc)](#Setup)
- [Example query](#Example-query)
- [Product definition](#Product-definition)
- [Quality layer](#Quality-layer)
- [Create and apply a good quality pixel mask](#Create-and-apply-a-good-quality-pixel-mask)
- [Plot and browse the data](#Plot-and-browse-the-data)

## Overview

This notebook demonstrates how to load and use NovaSAR-1 Radiometric Terrain Corrected (RTC) Gamma0 data from the [CSIRO NovaSAR Facility](https://research.csiro.au/cceo/novasar/).

These CEOS-ARD NovaSAR gamma-0 backscatter data are processed from NovaSAR-1 scenes using the [GAMMA software](https://gamma-rs.ch/gamma-software).

#### Using NovaSAR-1 backscatter data

An excellent introduction and overview to using SAR data is provided in the [CEOS Laymans SAR Interpretation Guide](https://ceos.org/ard/files/Laymans_SAR_Interpretation_Guide_3.0.pdf). This guide has also been converted to a set of set of Jupyter notebooks that you can download from https://github.com/AMA-Labs/cal-notebooks/tree/main/examples/SAR.

Synthetic Aperture Radar operates in the microwave range of the electromagnetic spectrum as an active pulse sent by the satellite and scattered by features on the Earth's surface. The return signal from the surface is measured at the satellite in terms of the signal intensity, phase and polarisation compared to the signal that was sent.

The SAR instrument on the NovaSAR-1 satellite operate in the S-band at approximately 9.4 cm wavelength. This means that it can "see" objects of about this size and larger, and smaller objects are relatively transparent. This makes Sentinel-1 more sensitive to vegetation structural types, sparse and low biomass vegetation, and surface water (smooth and wind affected). Compared to Sentinel-1 (C-band, 5.6 cm), NovaSAR S-band is less sensitive to tree canopies and has better ground penetration.

> The SAR signal responds to the orientation and scattering from surface features of comparable size or larger than the wavelength.
> - A bright backscatter value typically means the surface was orientated perpendicular to the signal incidence angle and most of the signal was reflected back to the satellite (direct backscatter)
> - A dark backscatter value means most of the signal was reflected away from the satellite (forward scattering) and typically responds to a smooth surface (relative to the wavelength) such as calm water or bare soil
> - Rough surfaces (relative to the wavelength) result in diffuse scattering where some of the signal is returned to the satellite.
> - Complex surfaces may result in volume scattering (scattering within a tree canopy) or double-bounce scattering (perpendicular objects such as buildings and structures)
> - The relative backscatter values of co-polarisation (HH, VV) and cross-polarisation (HV) measurements can provide information on the scattering characteristics of the surface features.

Using NovaSAR-1 backscatter data requires interpretation of the data for different surface features, including as these features change spatially or in time. It may also be necessary to carefully consider the incidence angle of the SAR signal relative to the surface features using the _incidence_angle_ band or the satellite direction metadata (descending = north to south; ascending = south to north).

#### Units and conversions
The `novasar_ceosard_*` data are given in _Amplitude_ units. Amplitude can be converted to _decibel (dB)_ or _intensity_, and vice-versa, with the following equations. Practical _Xarray_ examples are given below.

Amplitude to/from dB:
```
       dB = 20 * log10(intensity) + K
intensity = 10^((dB-K)/20)

where K is a calibration factor, which for NovaSAR-1 is -83 dB.
```

Intensity (or backscatter _power_) to/from Amplitude:
```
intensity = amplitude * amplitude
amplitude = sqrt(intensity)
```

Additional reference: https://forum.step.esa.int/t/what-stage-of-processing-requires-the-linear-to-from-db-command

## Set up

#### Imports

In [ ]:
# Common imports and settings
import os, sys, re
from pathlib import Path
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr
import numpy as np

# Datacube
import datacube
from datacube.utils.aws import configure_s3_access
import odc.geo.xr                             # https://github.com/opendatacube/odc-geo
from datacube.utils import masking            # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from dea_tools.plotting import display_map    # https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools

# Dask
import dask
from dask.distributed import Client, LocalCluster

# Basic plots
%matplotlib inline
# import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = [12, 8]

# Holoviews
# https://holoviz.org/tutorial/Composing_Plots.html
# https://holoviews.org/user_guide/Composing_Elements.html
import hvplot.xarray
import panel as pn
from datashader import reductions
from holoviews import opts

In [ ]:
# EASI defaults
# These are convenience functions so that the notebooks in this repository work in all EASI deployments (comment-out if not required)

repo = Path.home() / 'easi-notebooks'    # Change i s
if str(repo) not in sys.path:
    sys.path.append(str(repo))

# from easi_tools import EasiDefaults  # Convenience
from easi_tools import initialize_dask, xarray_object_size, heading  # Useful

#### EASI defaults

These default values are configured for each EASI instance. They help us to use the same training notebooks in each EASI instance. You may find some of the functions convenient for your work or you can easily override the values in your copy of this notebook.

In [ ]:
# TODO update for a NovaSAR product or set of NovaSAR products

# easi = EasiDefaults()

# family = 'sentinel-1'
# product = easi.product(family)   # 'sentinel1_grd_gamma0_20m'
# display(Markdown(f'Default {family} product for "{easi.name}": [{product}]({easi.explorer}/products/{product})'))

#### Dask cluster

Using a local _Dask_ cluster is a good habit to get into. It can simplify loading and processing of data in many cases, and it provides a dashboard that shows the loading/processing progress.

To learn more about _Dask_ see the set of [dask notebooks](https://github.com/csiro-easi/easi-notebooks/tree/main/html#dask-tutorials).

In [ ]:
# Local cluster
# cluster, client = initialize_dask(workers=4)  # EasiDefaults not configured for dev
# display(client)

# Damien's dask dashboard user path not available in dev (yet)
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
server = f'https://hub.dev.easi-eo.solutions'  # Or replace if not using EasiDefaults
user = os.environ.get('JUPYTERHUB_SERVICE_PREFIX')  # Current user
dask.config.set({"distributed.dashboard.link": f'{server}{user}' + "proxy/{port}/status"})  # port is evaluated by dask
display(client)

# Or use Dask Gateway - this may take a few minutes
# cluster, client = initialize_dask(use_gateway=True, workers=4)
# display(client)

#### ODC database

Connect to the ODC database. Configure the environment and low-level tools to read from AWS buckets.

In [ ]:
dc = datacube.Datacube()

# Access AWS "requester-pays" buckets
# This is necessary for reading data from most third-party AWS S3 buckets such as for Landsat and Sentinel-2
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

## Example query

Change any of the parameters in the `query` object below to adjust the location, time, projection, or spatial resolution of the returned datasets.

Use the Explorer interface to check the temporal and spatial coverage for each product.

In [ ]:
# Explorer link
# display(Markdown(f'See: {easi.explorer}/products/{product}'))

# # EASI defaults
# display(Markdown(f'#### Location: {easi.location}'))
# latitude_range = easi.latitude
# longitude_range = easi.longitude
# time_range = easi.time

# Or set your own latitude / longitude
# Australia GWW
# latitude_range = (-33, -32.6)
# longitude_range = (120.5, 121)
# time_range = ('2020-01-01', '2020-01-31')

# Mt Gambier
# https://explorer.dev.easi-eo.solutions/products/novasar_ceosard_scd_vv_hh_hv/datasets/a522f17a-8537-52e1-b574-d9ef290c151f
# https://novasar.dev.easi-eo.solutions/collections/novasar_ceosard_scd_vv_hh_hv/items/a522f17a-8537-52e1-b574-d9ef290c151f
product = 'novasar_ceosard_scd_vv_hh_hv'
longitude_range = (140.2, 141.0)
latitude_range = (-38.1, -37.5)
time_range = '2021-06-12'

products = [
    'novasar_ceosard_grd_hh',  # NovaSAR-1 CEOS-ARD GRD HH polarization
    'novasar_ceosard_grd_vv',  # NovaSAR-1 CEOS-ARD GRD VV polarization
    'novasar_ceosard_scd_hh',  # NovaSAR-1 CEOS-ARD SCD HH polarization
    'novasar_ceosard_scd_hh_hv',  # NovaSAR-1 CEOS-ARD SCD HH,HV polarizations
    'novasar_ceosard_scd_hv',  # NovaSAR-1 CEOS-ARD SCD HV polarization
    'novasar_ceosard_scd_vv',  # NovaSAR-1 CEOS-ARD SCD VV polarization
    'novasar_ceosard_scd_vv_hh',  # NovaSAR-1 CEOS-ARD SCD VV,HH polarizations
    'novasar_ceosard_scd_vv_hh_hv',  # NovaSAR-1 CEOS-ARD SCD VV,HH,HV polarizations
]

query = {
    'product': 'novasar_ceosard_scd_vv_hh_hv',       # Product name
    'measurements': ['vv', 'hh', 'hv', 'angle', 'scatteringarea', 'gammatosigmaratio', 'mask'],  # All bands for testing but using alias labels for consistency with S1
    'x': longitude_range,     # "x" axis bounds
    'y': latitude_range,      # "y" axis bounds
    'time': time_range,       # Any parsable date strings
}

# Convenience function to display the selected area of interest
display_map(longitude_range, latitude_range)

## Load data

In [ ]:
# Target xarray parameters
# - Select a set of measurements to load
# - output CRS and resolution
# - Usually we group input scenes on the same day to a single time layer (groupby)
# - Select a reasonable Dask chunk size (this should be adjusted depending on the
#   spatial and resolution parameters you choose
load_params = {
    'group_by': 'solar_day',                        # Scene grouping
    'dask_chunks': {'latitude':2048, 'longitude':2048},      # Dask chunks
}

# Load data
data = dc.load(**(query | load_params))
display(xarray_object_size(data))
display(data)

In [ ]:
# When happy with the shape and size of chunks, persist() the result
data = data.persist()

## Conversion and helper functions

In [ ]:
# These functions use numpy, which should be satisfactory for most notebooks.
# Calculations for larger or more complex arrays may require Xarray's "ufunc" capability.
# https://docs.xarray.dev/en/stable/examples/apply_ufunc_vectorize_1d.html
#
# Apply numpy.log10 to the DataArray
# log10_data = xr.apply_ufunc(np.log10, data)

def dn_to_decibel(da: 'xr.DataArray', K=0):
    """Return an array converted to dB values"""
    xx = da.where(da > 0, np.nan)  # Set values <= 0 to NaN
    xx = 20*np.log10(xx) + K
    xx.attrs.update({"units": "dB"})
    return xx

def decibel_to_dn(da: 'xr.DataArray', K=0):
    """Return an array converted to intensity values"""
    xx = np.power(10, (da-K)/20.0)
    xx.attrs.update({"units": "intensity"})
    return xx

def dn_to_amplitude(da: 'xr.DataArray'):
    """Return an array converted to amplitude values"""
    # TODO get scaling-factor from measurement (ODC database)
    xx = da / 14125.3754
    xx.attrs.update({"units": "amplitude"})
    return xx

def select_valid_time_layers(ds: 'xarray', percent: float = 5):
    """Select time layers that have at least a given percentage of valid data (e.g., >=5%)

    Example usage:
      selected = select_valid_time_layers(ds, percent=5)
      filtered == ds.sel(time=selected)
    """
    spatial_dims = ds.odc.spatial_dims
    return ds.count(dim=spatial_dims).values / (ds.sizes[spatial_dims[0]]*ds.sizes[spatial_dims[1]]) >= (percent/100.0)

# Examples to check that the intensity to/from dB functions work as expected
# xx = data.vv.isel(time=0,latitude=np.arange(0, 5),longitude=np.arange(0, 5))
# xx[0] = 0
# xx[1] = -0.001
# display(xx.values)
# yy = dn_to_decibel(xx)
# display(yy.values)
# zz = decibel_to_dn(yy)
# display(zz.values)

In [ ]:
# hvPlot convenience functions
def make_image(ds: 'xarray', frame_height=300, **kwargs):
    """Return a Holoviews DynamicMap (image) object that can be displayed or combined"""
    spatial_dims = ds.odc.spatial_dims
    defaults = dict(
        cmap="Greys_r",
        y = spatial_dims[0], x = spatial_dims[1],
        groupby = 'time',
        rasterize = True,
        geo = True,
        robust = True,
        frame_height = frame_height,
        clabel = ds.attrs.get('units', None),
    )
    defaults.update(**kwargs)
    return ds.hvplot.image(**defaults)

def rgb_image(ds: 'xarray', frame_height=300, **kwargs):
    """Return a Holoviews DynamicMap (RBG image) object that can be displayed or combined"""
    spatial_dims = ds.odc.spatial_dims
    defaults = dict(
        bands='band',
        y = spatial_dims[0], x = spatial_dims[1],
        groupby = 'time',
        rasterize = True,
        geo = True,
        robust = True,
        frame_height = frame_height,
    )
    defaults.update(**kwargs)
    return ds.hvplot.rgb(**defaults)


from bokeh.models.tickers import FixedTicker
def mask_image(ds: 'xarray', frame_height=300, **kwargs):
    """Return a Holoviews DynamicMap (RBG image) object that can be displayed or combined"""
    # NovaSAR ARD mask
    color_def = [
        (0,  '#ff0004', 'InvalidData'),   # red
        (1,  '#eeeeee', 'ValidData'),   # light grey
        (5, '#ff52ff', 'Layover'),   # cyan
        (17,  '#774c0b', 'Shadow'),   # brown
    ]
    color_val = [x[0] for x in color_def]
    color_hex = [x[1] for x in color_def]
    color_txt = [f'{x[0]:2d}: {x[2]}' for x in color_def]
    color_lim = (min(color_val), max(color_val) + 1)
    bin_edges = color_val + [max(color_val) + 1]
    # bin_range = (color_val[0] + 0.5, color_val[-1] + 0.5)  # No idea why (0.5,11.5) works and (0,11) or (0,12) do not

    # Image options
    defaults = {
        'aggregator': reductions.mode(),
        'cmap': color_hex,
        'clim': color_lim,
        'colorbar': True,
        'frame_height': frame_height,
    }
    # Colorbar options for categories
    extra_opts = {
        'color_levels': bin_edges,
        'colorbar_opts': {
            'major_label_overrides': dict(zip(color_val, color_txt)),
            'major_label_text_align': 'left',
            'ticker': FixedTicker(ticks=color_val),
        },
    }
    defaults.update(**kwargs)
    return make_image(ds, **defaults).options(opts.Image(**extra_opts))  #.hist(bins=bin_edges)

In [ ]:
# Optional time layer filter

# selected = select_valid_time_layers(data.vv, 10)  # Exclude time layers with less than 10% valid data
# data = data.sel(time=selected).persist()

In [ ]:
# Add dB and amplitude values to the dataset

data['vv_db'] = dn_to_decibel(data.vv, K=-83).persist()
data['hh_db'] = dn_to_decibel(data.hh, K=-83).persist()
data['hv_db'] = dn_to_decibel(data.hv, K=-83).persist()

data['vv_amp'] = dn_to_amplitude(data.vv).persist()
data['hh_amp'] = dn_to_amplitude(data.hh).persist()
data['hv_amp'] = dn_to_amplitude(data.hv).persist()

data.mask.persist()

## Plot the data

> Note the different data ranges for plotting (`clim`) between `vv`, `vh`, _intensity_ and _dB_.

- Stronger co-polarisation (VV) indicates direct backscatter while stronger cross-polarisation (VH) may indicate a complex surface or volume scattering.
- Intensity data are linear-scaled so can tend to disciminate across a range of backscatter returns.
- Decibel data are log-scaled so can tend to discriminate high and low backscatter returns.

In [ ]:
# VV, HH and HV (amplitude and dB) and Angle hvPlots

vv_plot = make_image(data.vv_amp, title='VV (amplitude)', clim=(0, 0.5))
hh_plot = make_image(data.hh_amp, title='HH (amplitude)', clim=(0, 0.5))
hv_plot = make_image(data.hv_amp, title='HV (amplitude)', clim=(0, 0.25))

vv_db_plot = make_image(data.vv_db, title='VV (dB)', clim=(-20, -5))
hh_db_plot = make_image(data.hh_db, title='HH (dB)', clim=(-20, -5))
hv_db_plot = make_image(data.hv_db, title='HV (dB)', clim=(-35, -10))

ia_plot = make_image(data.angle, title='Incidence angle')
sa_plot = make_image(data.scatteringarea, title='Scattering area')
gs_plot = make_image(data.gammatosigmaratio, title='Gamma to sigma ratio')
mask_plot = mask_image(data.mask, title='Mask')

In [ ]:
# Arrange plots with linked axes and time slider. Adjust browser window width if required.

layout = pn.panel(
    (vv_plot + hh_plot + hv_plot + \
     vv_db_plot + hh_db_plot + hv_db_plot + \
     ia_plot + sa_plot + gs_plot + mask_plot).cols(3),
    widget_location='top',
)
print(layout)  # Helpful to see how the hvplot is constructed
layout

# -------------

> **The remainder of this notebook is yet to be updated for NovaSAR**

## Plot a histogram of the dB data

A histogram can help separate water from land features. Here we show a histogram for the _HV (db)_ channel for all time layers.
- If the histogram shows two clear peaks then a value between the peaks could be used as a water / land threshold
- If not then try selected time layers, a different area of interest, or other channels or combinations.

In [ ]:
# vals, bins, hist_plot = data.hv_db.plot.hist(bins=np.arange(-30, 0, 1), color='red')  # Matplotlib
hist_plot = data.hv_db.hvplot.hist(bins=np.arange(-30, 0, 1), color='red', title='Combined times', height=400)  # hvPlot

print(hist_plot)  # Helpful to see how the hvplot is constructed
hist_plot

## Make an RGB image

A common strategy to create an RGB colour composite image for SAR data from two channels is to use the ratio of the channels to represent the third colour. Here we choose

To create an RGB colour composite image we can use the ratio of VH and VV to represent a third channel. Here we choose
- Red = VHV ... complex scattering
- Green = VV ... direct scattering
- Blue = VH/VV ... relatively more complex than direct

In [ ]:
# Add the vh/vv band to represent 'blue'
data['vh_vv'] = data.vh / data.vv

# Scale the measurements by their median so they have a similar range for visualization
spatial_dims = data.odc.spatial_dims
data['vh_scaled'] = data.vh / data.vh.median(dim=spatial_dims).persist()
data['vv_scaled'] = data.vv / data.vv.median(dim=spatial_dims).persist()
data['vh_vv_scaled'] = data.vh_vv / data.vh_vv.median(dim=spatial_dims).persist()

# odc-geo function
rgb_data = data.odc.to_rgba(bands=['vh_scaled','vv_scaled','vh_vv_scaled'], vmin=0, vmax=2)

In [ ]:
# As subplots
# rgb_plot = rgb_image(
#     rgb_data,
# ).layout().cols(4)

# As movie. Select "loop" and use "-" button to adjust the speed to allow for rendering. After a few cycles the images should play reasonably well.
rgb_plot = rgb_image(
    rgb_data,
    precompute = True,
    widget_type='scrubber', widget_location='bottom',
    frame_height = 500,
)

print(rgb_plot)  # Helpful to see how the hvplot is constructed
rgb_plot

## Export to Geotiffs

Recall that to write a dask dataset to a file requires the dataset to be `.compute()`ed. This may result in a large memory increase on your JupyterLab node if the area of interest is large enough, which in turn may kill the kernel. If so then skip this step, choose a smaller area or find a different way to export data.

In [ ]:
# Make a directory to save outputs to
target = Path.home() / 'output'
if not target.exists(): target.mkdir()

def write_band(ds, varname):
    """Write the variable name of the xarray dataset to a Geotiff file for each time layer"""
    for i in range(len(ds.time)):
        date = ds[varname].isel(time=i).time.dt.strftime('%Y%m%d').data
        fname = f'{target}/example_sentinel-1_{varname}_{date}.tif'
        single = ds[varname].isel(time=i).compute()
        single.odc.write_cog(
            fname=fname,
            overwrite=True,
        )
        print(f'Wrote: {fname}')
        
write_band(data, 'vv')
write_band(data, 'vh')